#  Agentic RAG with Shivaay

This notebook demonstrates how to simulate an agent-based Retrieval-Augmented Generation (RAG) workflow using the Shivaay API. It walks through the process of:
- Breaking down a complex query into subtasks,
- Generating search queries for each,
- Simulating document retrieval,
- And finally using Shivaay to synthesize an answer from all gathered context.

###  Step 1: Setup – Import Dependencies
- Imports required libraries:
  - `requests` for API calls to Shivaay.
  - `time` for potential throttling between API requests.

In [8]:
import requests
import time

###  Step 2: Configure Shivaay API
- Sets your Shivaay API key and endpoint.
- Defines a helper function `call_shivaay()` to make POST requests.
- Accepts parameters like `system_prompt`, `user_input`, `temperature`, and `max_tokens`.


In [9]:
API_KEY = "YOUR-API-KEY"
API_URL = "https://api.futurixai.com/api/shivaay/v1/chat/completions"

def call_shivaay(messages, temperature=0.3, max_tokens=200):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    payload = {
        "model": "shivaay",
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


###  Step 3: Planning Agent
- Shivaay is prompted to break down a complex user query into a list of smaller subtasks.
- Mimics how an autonomous agent would reason about a problem before retrieving information.


In [10]:
def planning_agent(user_question):
    messages = [
        {"role": "system", "content": "You're a helpful planner AI. Break down the user question into logical steps or subtasks to find an accurate answer."},
        {"role": "user", "content": user_question}
    ]
    return call_shivaay(messages)


###  Step 4: Search Agent
- For each subtask, Shivaay generates 2–3 effective search queries.
- These simulate how a retrieval-focused agent might think and formulate search strings.
- Queries are parsed and returned as a clean list.

In [11]:
def search_agent(subtask):
    messages = [
        {"role": "system", "content": "Generate 2-3 highly relevant web search queries based on the subtask provided."},
        {"role": "user", "content": subtask}
    ]
    raw = call_shivaay(messages)
    return [q.strip("- ").strip() for q in raw.split("\n") if q.strip()]


###  Step 5: Retrieval Agent (Simulated)
- Instead of using a real search engine or vector DB, this step mocks retrieved results using predefined text.
- Each search query maps to a simulated paragraph of context.
- All paragraphs are combined for final use.

In [12]:
def retrieval_agent(fake_results):
    return "\n".join(fake_results)


###  Step 6: Reasoning Agent
- Combines all context and sends it back to Shivaay with the original user question.
- Shivaay answers the question strictly using the retrieved context.
- Demonstrates the reasoning capability of the agent pipeline.


In [13]:
def reasoning_agent(question, context):
    messages = [
        {"role": "system", "content": f"Use the following context to answer the user's question:\n\n{context}"},
        {"role": "user", "content": question}
    ]
    return call_shivaay(messages, temperature=0.2)


###  Step 7: Output Final Answer
- Prints the complete answer generated by Shivaay.
- Optionally previews part of the context used for transparency.

In [14]:
user_question = input("💬 Enter your complex question: ")

print("\n🧠 Planning agent thinking...")
plan = planning_agent(user_question)
print("✅ Plan:\n", plan)

subtasks = [line.strip("- ").strip() for line in plan.split("\n") if line.strip()]

all_contexts = []

for i, task in enumerate(subtasks):
    print(f"\n🔍 Searching for subtask {i+1}: {task}")
    queries = search_agent(task)
    print("Generated Queries:", queries)

    # Simulate retrieval (replace with actual web scraping if needed)
    mock_results = [f"Result for '{q}' - Lorem ipsum content about {q}" for q in queries]
    context = retrieval_agent(mock_results)
    all_contexts.append(context)

print("\n🧠 Reasoning agent compiling final answer...")
final_context = "\n\n".join(all_contexts)
answer = reasoning_agent(user_question, final_context)

print("\n📝 Final Answer:\n", answer)


💬 Enter your complex question: How can renewable energy sources like solar and wind be integrated into national power grids, and what are the main challenges faced during this transition?

🧠 Planning agent thinking...
✅ Plan:
 Integrating renewable energy sources such as solar and wind into national power grids involves several steps and faces various challenges. Here's a breakdown of the process and the main challenges:

### Steps for Integration

1. **Assessment and Planning:**
   - Conduct feasibility studies to determine the potential for renewable energy generation.
   - Plan the integration of renewable sources based on geographical suitability, existing infrastructure, and future energy demands.

2. **Infrastructure Development:**
   - Upgrade or build new transmission lines to accommodate the fluctuating output from renewable sources.
   - Install necessary equipment for grid stabilization, such as energy storage systems and smart grid technologies.

3. **Policy and Regulatory 